In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from utils import *

from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset, Dataset
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [3]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=250)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1.mean(),
        'precision': precision.mean(),
        'recall': recall.mean()
    }

def acc_at_k(y_true, y_pred, k=2):
    y_true = torch.tensor(y_true) if type(y_true) != torch.Tensor else y_true
    y_pred = torch.tensor(y_pred) if type(y_pred) != torch.Tensor else y_pred
    total = len(y_true)
    y_weights, y_idx = torch.topk(y_true, k=k, dim=-1)
    out_weights, out_idx = torch.topk(y_pred, k=k, dim=-1)
    correct = torch.sum(torch.eq(y_idx, out_idx) * y_weights)
    acc = correct / total
    return acc.item()

def CEwST_loss(logits, target, reduction='mean'):
    """
    Cross Entropy with Soft Target (CEwST) Loss
    :param logits: (batch, *)
    :param target: (batch, *) same shape as logits, each item must be a valid distribution: target[i, :].sum() == 1.
    """
    logprobs = torch.nn.functional.log_softmax(logits.view(logits.shape[0], -1), dim=1)
    batchloss = - torch.sum(target.view(target.shape[0], -1) * logprobs, dim=1)
    if reduction == 'none':
        return batchloss
    elif reduction == 'mean':
        return torch.mean(batchloss)
    elif reduction == 'sum':
        return torch.sum(batchloss)
    else:
        raise NotImplementedError('Unsupported reduction mode.')

def compute_metrics_w_soft_target(pred):
    labels = pred.label_ids
    preds = pred.predictions
    acc = acc_at_k(labels, preds, k=2)
    return {
        'accuracy': acc,
    }

class Trainer_w_soft_target(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0]
        loss = CEwST_loss(logits, labels)
        if return_outputs:
            return loss, outputs
        return loss
    
class DefaultCollator:
    def __init__(self):
        pass
    def __call__(self, batch):
        return torch.utils.data.dataloader.default_collate(batch)

In [4]:
MODEL_NAMES = ['bert-base-uncased'] #, 'xlnet-base-cased']

In [5]:
from sklearn.datasets import fetch_20newsgroups

def get_20NG_test_dataset():
    cats = [
        'talk.politics.mideast',                                # Wolrd 0
        'rec.sport.hockey', 'rec.sport.baseball',               # Sports 1
        # 'misc.forsale',                                       # Business 2
        'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', # Sci/Tech 3
    ]

    dataset = fetch_20newsgroups(
        subset='all',
        categories=cats,
        remove=('headers', 'footers', 'quotes')
    )

    df = pd.DataFrame([dataset.data, dataset.target]).T
    df.rename(columns={0:'text', 1: 'label'}, inplace=True)

    mapper = {
        0: 1,
        1: 1,
        2: 3,
        3: 3,
        4: 3,
        5: 3,
        6: 0,
    }

    df.label = df.label.map(mapper)
    df.text = df.text.replace('\n', ' ', regex=True).str.strip()

    test_dataset = Dataset.from_pandas(df)
    
    return test_dataset

In [6]:
use_pretrain = True
soft_target = False

for t in ['ORIG', 'INV', 'SIB-mix', 'INVSIB']:
    for MODEL_NAME in MODEL_NAMES:
        
        eval_only = False
        
        checkpoint = 'pretrained/' + MODEL_NAME
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)    
        
        if t == 'ORIG':
            train_dataset = load_dataset('ag_news')['train']
            checkpoint += '-ag_news-ORIG'
            if use_pretrain and os.path.exists(checkpoint):
                print('loading {}...'.format(checkpoint))
                MODEL_NAME = checkpoint
                eval_only = True
        if t == 'INV':
            text = npy_load("./assets/AG_NEWS/topic/INV/text.npy")
            label = npy_load("./assets/AG_NEWS/topic/INV/label.npy")
            df = pd.DataFrame({'text': text, 'label': label})
            df.text = df.text.astype(str)
            df.label = df.label.astype(int)
            train_dataset = Dataset.from_pandas(df)
            checkpoint += '-ag_news-INV'
            if use_pretrain and os.path.exists(checkpoint):
                print('loading {}...'.format(checkpoint))
                MODEL_NAME = checkpoint
                eval_only = True
        if t == 'SIB-mix':
            text = npy_load("./assets/AG_NEWS/topic/SIB-mix/text.npy")
            label = npy_load("./assets/AG_NEWS/topic/SIB-mix/label.npy")
            df = pd.DataFrame({'text': text, 'label': label.tolist()})
            df.text = df.text.astype(str)
            df.label = df.label.map(lambda y: np.array(y))
            train_dataset = Dataset.from_pandas(df)    
            checkpoint += '-ag_news-SIB-mix'
            if use_pretrain and os.path.exists(checkpoint):
                print('loading {}...'.format(checkpoint))
                MODEL_NAME = checkpoint  
                eval_only = True
        if t == 'INVSIB':
            text = npy_load("./assets/AG_NEWS/topic/INVSIB/text.npy")
            label = npy_load("./assets/AG_NEWS/topic/INVSIB/label.npy")
            df = pd.DataFrame({'text': text, 'label': label.tolist()})
            df.text = df.text.astype(str)
            df.label = df.label.map(lambda y: np.array(y))
            train_dataset = Dataset.from_pandas(df)    
            checkpoint += '-ag_news-INVSIB'
            if use_pretrain and os.path.exists(checkpoint):
                print('loading {}...'.format(checkpoint))
                MODEL_NAME = checkpoint 
                eval_only = True
                
        dataset_dict = train_dataset.train_test_split(
            test_size = 0.1,
            train_size = 0.9,
            shuffle = True
        )
        train_dataset = dataset_dict['train']
        eval_dataset = dataset_dict['test']
        test_dataset = load_dataset('ag_news')['test']
        test_dataset_20NG = get_20NG_test_dataset()
        
        # # reduce training time
        # n = 100
        # train_dataset = Dataset.from_dict(train_dataset[:n])
        # eval_dataset = Dataset.from_dict(eval_dataset[:n])
        # test_dataset = Dataset.from_dict(test_dataset[:n])

        model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=4).to(device)
                
        train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
        eval_dataset = eval_dataset.map(tokenize, batched=True, batch_size=len(eval_dataset))
        test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
        test_dataset_20NG = test_dataset_20NG.map(tokenize, batched=True, batch_size=len(test_dataset_20NG))
        train_dataset.rename_column_('label', 'labels')
        eval_dataset.rename_column_('label', 'labels')
        test_dataset.rename_column_('label', 'labels')
        test_dataset_20NG.rename_column_('label', 'labels')
        train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
        eval_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
        test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
        test_dataset_20NG.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
        
        if len(np.array(train_dataset['labels']).shape) > 1:
            soft_target = True

        train_batch_size = 3
        eval_batch_size = 32
        num_epoch = 10
        max_steps = int((len(train_dataset) * num_epoch) / train_batch_size)
        
        training_args = TrainingArguments(
            output_dir=checkpoint,
            overwrite_output_dir=True,
            max_steps=max_steps,
            save_steps=int(max_steps / 10),
            save_total_limit=1,
            per_device_train_batch_size=train_batch_size,
            per_device_eval_batch_size=eval_batch_size,
            warmup_steps=int(max_steps / 100),
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=int(max_steps / 100),
            load_best_model_at_end=True,
            metric_for_best_model="loss",
            greater_is_better=False,
            evaluation_strategy="steps",
            label_names=['World', 'Sports', 'Business', 'Sci/Tech']
        )
        
        if soft_target:
            trainer = Trainer_w_soft_target(
                model=model,
                args=training_args,
                compute_metrics=compute_metrics_w_soft_target,
                train_dataset=train_dataset,
                eval_dataset=eval_dataset,
                data_collator=DefaultCollator(),
                callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
            )
        else: 
            trainer = Trainer(
                model=model,
                args=training_args,
                compute_metrics=compute_metrics,
                train_dataset=train_dataset,
                eval_dataset=test_dataset,
                callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
            )

        if not eval_only:
            trainer.train()
            
        trainer.compute_metrics = compute_metrics
        
        # test with ORIG data
        trainer.eval_dataset = test_dataset
        out_orig = trainer.evaluate()
        print('ORIG \n', out_orig)
        
        # test with 20NG data
        trainer.eval_dataset = test_dataset_20NG
        out_20NG = trainer.evaluate()
        print('20NG \n', out_20NG)

Using custom data configuration default
Reusing dataset ag_news (C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)
Loading cached split indices for dataset at C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a\cache-d83824437fa96733.arrow and C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a\cache-19c3de4609a5743e.arrow


loading pretrained/bert-base-uncased-ag_news-ORIG...


Using custom data configuration default
Reusing dataset ag_news (C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)
Loading cached processed dataset at C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a\cache-e59b4416e2d48217.arrow
Loading cached processed dataset at C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a\cache-671c7189356b54e5.arrow
Loading cached processed dataset at C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a\cache-4a398149fbe9330d.arrow


C:\Users\Fabrice\AppData\Local\Continuum\anaconda3\envs\python38\lib\site-packages\datasets\arrow_dataset.py:850: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


ORIG 
 {'eval_loss': 0.5152392387390137, 'eval_accuracy': 0.8953947368421052, 'eval_f1': 0.8953876564071228, 'eval_precision': 0.9005043147915526, 'eval_recall': 0.8953947368421054, 'eval_runtime': 116.1718, 'eval_samples_per_second': 65.42}


...

Loading cached processed dataset at C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a\cache-4a398149fbe9330d.arrow


ORIG 
 {'eval_loss': 28.212501525878906, 'eval_accuracy': 0.9, 'eval_f1': 0.9007377666445252, 'eval_precision': 0.9047491703841797, 'eval_recall': 0.8999999999999999, 'eval_runtime': 115.645, 'eval_samples_per_second': 65.718}
20NG 
 {'eval_loss': 31.028812408447266, 'eval_accuracy': 0.831517792302106, 'eval_f1': 0.6227493445608775, 'eval_precision': 0.6326871288115772, 'eval_recall': 0.6140993670315309, 'eval_runtime': 107.1969, 'eval_samples_per_second': 64.228}
loading pretrained/bert-base-uncased-ag_news-INVSIB...


Using custom data configuration default
Reusing dataset ag_news (C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)


Loading cached processed dataset at C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a\cache-4a398149fbe9330d.arrow


ORIG 
 {'eval_loss': 23.542787551879883, 'eval_accuracy': 0.9001315789473684, 'eval_f1': 0.899759351408246, 'eval_precision': 0.9032745056421649, 'eval_recall': 0.9001315789473685, 'eval_runtime': 116.761, 'eval_samples_per_second': 65.09}
20NG 
 {'eval_loss': 26.4996280670166, 'eval_accuracy': 0.8104575163398693, 'eval_f1': 0.5989796589956652, 'eval_precision': 0.6113622084996353, 'eval_recall': 0.587867818720505, 'eval_runtime': 105.9039, 'eval_samples_per_second': 65.012}
